Madi from Image Reserach center

In [ ]:
# ###LDA Object Detection RSNA Database

# #!conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch


# #Please install all libraries
# !pip install opencv-python
# !pip install unet
# !pip install scipy
# !pip install seaborn
# !pip install h5py
# !pip install utils
# !pip install pandas
# !pip install matplotlib
# !pip install nibabel
# !pip install SimpleITK
# !pip install scikit-plot
# !pip install tqdm
# !pip install scikit-image
# !pip install ipywidgets
# !pip install torchvision
# !pip install torchio
# !pip install albumentations
# !pip install -q pydicom 
# !pip install -q imgaug
# !pip install -U albumentations
# !pip install minio
# !pip install python-dotenv
# !pip install boto3
# !pip install plotly
#!pip install mlxtend
#!pip install detecto

In [ ]:
from __future__ import division, print_function, absolute_import

import math
import time
import sys
import cv2
import os
import scipy
import enum
import random
import h5py
import zipfile
import itertools
import json
import seaborn as sns
import pydicom as dcm
import numpy as np
import pandas as pd
import multiprocessing
import scipy.ndimage.morphology as morphology
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import matplotlib.pyplot as plt
import matplotlib.image as im
import matplotlib.image as img
import nibabel as nib
import SimpleITK as sitk
import plotly.express as px
import plotly.graph_objects as go
import pydicom 
from itertools import product

from matplotlib.patches import Rectangle
from scipy import stats
from scipy import ndimage
from pathlib import Path
from pathlib import Path
#from unet import UNet
from PIL import Image
from os import listdir
from os.path import join, basename, isdir
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from tqdm.notebook import trange, tqdm, tqdm_notebook
from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid

from pydicom import dcmread
from glob import glob
from skimage.measure import label
from skimage.transform import resize
from skimage import io, measure
from nibabel.testing import data_path
from ipywidgets import interact, fixed
from IPython import display
from tqdm.notebook import tqdm
from pathlib import Path

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchio as tio
import torch.utils.data
import torchvision
import torchvision.utils
import torchvision.models as models
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF


from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from sklearn.model_selection import train_test_split
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data.sampler import SequentialSampler
from torchvision import transforms
from torchvision import transforms as T
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torchio.transforms import HistogramStandardization
from torch.autograd import Variable

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A
from albumentations import (
    Resize,
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)


print('TorchIO version:', tio.__version__)
print('Torch version:', tio.__version__)

In [ ]:
torch.cuda.is_available()

##  Setup Data Directory

In [ ]:
#!nvidia-smi

In [ ]:
#!pwd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#os.chdir('/workspace/LDA_Object_Detection/')
images_path = 'D:/Datasets/LDA/RSNA/stage_2_train_images/'
print(os.getcwd())
print(images_path)

In [ ]:
len(os.listdir(images_path))

In [ ]:
print(images_path[:5])

## Excel Files

In [ ]:
train_labels = pd.read_csv(os.path.join('stage_2_train_labels.csv'))
class_labels = pd.read_csv(os.path.join('stage_2_detailed_class_info.csv'))

In [ ]:
train_labels

In [ ]:
class_labels

In [ ]:
train_labels.info()

In [ ]:
## we can see that all the null column values are with Target 0 indicating that those patients do not have penumonia
train_labels[train_labels.isnull().any(axis=1)].Target.value_counts()

In [ ]:
## we can see that all the non null column values are with Target 1 indicating that those patients have pneumonia
train_labels[~train_labels.isnull().any(axis=1)].Target.value_counts()

In [ ]:
train_labels = train_labels.drop_duplicates("patientId")
# train_labels

#### PatientId 
A patientId. Each patientId corresponds to a unique image

#### Class 
Have three values depending what is the current state of the patient's lung: 'No Lung Opacity / Not Normal', 'Normal' and 'Lung Opacity'.

In [ ]:
# print('First five rows of Training set:\n', train_labels.head())
# print(train_labels.iloc[0])

In [ ]:
# print(train_labels.iloc[4])

#### Some information about the data field present in the 'stage_2_train_labels.csv' are:

-patientId - A patientId. Each patientId corresponds to a unique image (which we will see a little bit later)

-x - The upper-left x coordinate of the bounding box

-y - The upper-left y coordinate of the bounding box

-width - The width of the bounding box

-height - The height of the bounding box

-Target - The binary Target indicating whether this sample has evidence of pneumonia or not.

In [ ]:
# Number of duplicates in patientId:
print('Number of unique patientId are: {}'.format(train_labels['patientId'].nunique()))

In [ ]:
bounding_box = train_labels.groupby('patientId').size().to_frame('number_of_boxes').reset_index()
train_labels = train_labels.merge(bounding_box, on = 'patientId', how = 'left')
print('Number of patientIds per bounding box in the dataset: ')
(bounding_box.groupby('number_of_boxes').size().to_frame('number_of_patientId').reset_index().set_index('number_of_boxes').sort_values(by = 'number_of_boxes'))

#### Classes Information/ Number of Target =1

In [ ]:
print('Original dataframe shape:', train_labels.shape)

train_labels_pos = pd.DataFrame(columns=['patientId', 'x', 'y', 'width', 'height'])
k = 0
for i in range(len(train_labels)):
    if train_labels.loc[i]['Target'] == 1:
        train_labels_pos.loc[k] = train_labels.loc[i]
        k += 1

print('Positive instances dataframe shape:', train_labels_pos.shape)
train_paths = [os.path.join(images_path, image[0]) for image in train_labels_pos.values]

In [ ]:
train_labels_pos.head()

In [ ]:
train_labels['Target'].value_counts()

# Visualization of the images and the areas of inflammation

In [ ]:
def imshow(num_to_show=9):
    
    plt.figure(figsize=(20,20))
    
    for i in range(num_to_show):
        plt.subplot(3, 3, i+1)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        
        img_dcm = dcmread(f'{train_paths[i+20]}.dcm')
        img_np = img_dcm.pixel_array
        plt.imshow(img_np, cmap='bone')
        print(img_np.shape)

imshow()

In [ ]:
def show_image_with_bboxes(num_to_show=9):
    plt.figure(figsize=(20,20))
    
    for i in range(num_to_show):
        plt.subplot(3, 3, i+1)
        plt.grid(False)
        plt.xticks([])
        plt.yticks([])
        
        id_= np.random.choice(train_labels_pos['patientId'].values)

        current_axis = plt.gca()
        img=pydicom.read_file(os.path.join(images_path,id_+'.dcm')).pixel_array
        plt.imshow(img,cmap='bone')
        print(img.shape)


        current_axis = plt.gca()
        boxes=train_labels_pos[['x','y','width','height']][train_labels_pos['patientId']==id_].values
        for box in boxes:
            x=box[0]
            y=box[1]
            w=box[2]
            h=box[3]
            current_axis.add_patch(plt.Rectangle((x, y), w, h, color='red', fill=False, linewidth=3)) 
        
show_image_with_bboxes()

# Visualization of one image with area of inflammation

In [ ]:
def parse_one_annot(data, patient_id):
    boxes_array = data[data["patientId"] == patient_id][["x", "y", "width", "height"]].values
    print(boxes_array.dtype)
    return boxes_array

In [ ]:
def show_one_with_bbox(id_):
    plt.figure(figsize=(20,20))
    current_axis = plt.gca()
    img=pydicom.read_file(os.path.join(images_path,id_+'.dcm')).pixel_array
    plt.imshow(img,cmap='bone')

    current_axis = plt.gca()
    boxes=train_labels_pos[['x','y','width','height']][train_labels_pos['patientId']==id_].values
    for box in boxes:
        x=box[0]
        y=box[1]
        w=box[2]
        h=box[3]
        current_axis.add_patch(plt.Rectangle((x, y), w, h, color='red', fill=False, linewidth=3)) 

In [ ]:
id_= np.random.choice(train_labels_pos['patientId'].values)
print('Id', id_)
print('Bboxes', parse_one_annot(train_labels_pos, id_))
show_one_with_bbox(id_)

# Preparing data for training. Dataset class. Tranformations

In [ ]:
class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['patientId'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        
    def __getitem__(self, index):
        # load images and bounding boxes   
        image_id = self.image_ids[index]
        records = self.df[self.df['patientId'] == image_id]
        
        img_path = os.path.join(self.image_dir, image_id)
        img=pydicom.read_file(os.path.join(img_path +'.dcm')).pixel_array
        img=img.astype(np.double)
        img /= 255.0  
        
        # BB cordinates
        
        records = records.astype({"x": float, "y": float, "height": float, "width": float})
        boxes = records[['x', 'y', 'width', 'height']].values
        
    
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

        # Count the area and convert to tensor
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]) 
        area = torch.as_tensor(area, dtype=torch.float32)                                  
        
        # We have only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)       

        
        
        # Bonding Box is in dictionary format
        target = {}    
        target['boxes'] = boxes
        target['labels'] = labels
        target['patientId'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd
                      
        if self.transforms:
            sample = {
                'image': img,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            img = sample['image']
            target['boxes'] = torch.stack(tuple(map(torch.FloatTensor, zip(*sample['bboxes'])))).permute(1, 0)
            #target['boxes'] = torch.stack(list((map(torch.tensor, target['boxes'])))).type(torch.float32)

        return img, target
    
    # return the number of dataset
    def __len__(self):   #-> int:
        #return self.image_ids.shape[0]  
        return len(self.image_ids)


# Transforms

In [ ]:
# Albumentations
#https://albumentations.ai/docs/examples/example_bboxes2/
#conda install -c conda-forge albumentations


def get_train_transform():
    return A.Compose([
        #Resize(300,  300),
#         A.Flip(0.5),
#         A.RandomRotate90(0.5),
#         MotionBlur(p=0.2),
#         MedianBlur(blur_limit=3, p=0.1),
#         Blur(blur_limit=3, p=0.1),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

# Split Dataset

In [ ]:
#input_folder = '../input/rsna-pneumonia-detection-2018/input'
images_folder = r'D:\Datasets\LDA\RSNA\stage_2_train_images'

image_ids = train_labels_pos['patientId'].unique()
train_ids = image_ids[:-300]
valid_ids = image_ids[-300:]

print(f"Training instance: {len(train_ids)}")
print(f"Validation instances: {len(valid_ids)}")

valid_df = train_labels_pos[train_labels_pos['patientId'].isin(valid_ids)]
train_df = train_labels_pos[train_labels_pos['patientId'].isin(train_ids)]

print('Train dataframe shape:', train_df.shape)
print('Valid dataframe shape:', valid_df.shape)
    
train_dataset = PneumoniaDataset(train_df, images_folder, get_train_transform())
valid_dataset = PneumoniaDataset(valid_df, images_folder, get_valid_transform())
print('train_dataset and valid_dataset are loaded :)')   
print("We have: {} training examples and {} validation examples".format(len(train_dataset), len(valid_dataset)))

# Check the custom dataset

In [ ]:
train_dataset = PneumoniaDataset(train_df, images_folder, get_train_transform())

train_dataset[1][0]
#train_dataset.__getitem__(10)

In [ ]:
train_dataset = PneumoniaDataset(train_df, images_folder, get_train_transform())
image = iter(train_dataset)
img, targets = next(image)
print(f'Tensor:{img}, Label:{label}')
img = np.transpose(img, (1, 2, 0))
plt.imshow(img)

In [ ]:
fig, axis = plt.subplots(2,2, figsize=(9,9))
for i in range(2):
    for j in range(2):
        random_index = np.random.randint(0, 240)
        x_ray, label = train_dataset[random_index]
        axis[i][j].imshow(x_ray[0], cmap="bone")
        axis[i][j].set_title(label)

# Data Loaders

In [ ]:
# collate_fn needs for batch

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# def collate_fn(batch):
#     return batch

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn) #, num_workers=2, collate_fn=collate_fn)
valid_data_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn) #, num_workers=2, collate_fn=collate_fn)

# Check dataloader

In [ ]:
# batch = iter(train_data_loader)
# images, targets = next(batch)
# image_grid = torchvision.utils.make_grid(images[:4])
# image_np = image_grid.numpy()
# img = np.transpose(image_np, (1, 2, 0))
# plt.imshow(img)

In [ ]:
images, targets = next(iter(train_data_loader))

In [ ]:
next(iter(train_data_loader))

In [ ]:
# ###https://stackoverflow.com/questions/5558418/list-of-dicts-to-from-dict-of-lists

# # DataLoader is iterable over Dataset
# for images, targets in train_data_loader:
#     images = list(image.to(device) for image in images)
#     #print(targets)
    
#     targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#     #targets = [{k: v for k, v in t.items()} for t in targets]
#     #targets = {k: v.to(device) for k, v in targets.items()}
#     print(targets)
        
#     ##Convert DL = targets
    
# #     targets = [dict(zip(targets,t)) for t in zip(*targets.values())]
    
# #     print(targets)

In [ ]:
# # Checking

# boxes = targets[1]['boxes'].cpu().numpy().astype(np.int64)
# print(boxes.shape)

# Helper Funtions in Training

In [ ]:
def calculate_image_precision(gts, preds, thresholds = (0.5, ), form = 'coco') -> float:
    # https://www.kaggle.com/sadmanaraf/wheat-detection-using-faster-rcnn-train
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision


def calculate_iou(gt, pr, form='pascal_voc') -> float:
    # https://www.kaggle.com/sadmanaraf/wheat-detection-using-faster-rcnn-train
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area


def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    # https://www.kaggle.com/sadmanaraf/wheat-detection-using-faster-rcnn-train
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1
    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx

def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    # https://www.kaggle.com/sadmanaraf/wheat-detection-using-faster-rcnn-train
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1
        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

# Train and Validate functions

In [ ]:
def train(dataloader, lr_scheduler, model, optimizer, 
          device, epoch, loss_hist, itr):
    model.train()
    start = time.time()
    loss_hist.reset()
    for images, targets in dataloader:
        
        images = list(image.to(device, dtype=torch.float) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
#         print('Targets has: ',  targets)
#         print('images has: ',  images)
        
#         print('Targets has: ', targets[0].keys())
#         print("my image  : ",images[0])

        loss_dict = model(images, targets)


        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 100 == 0:                                                     # if itr % 50 == 0:
            print(f"Epoch #{epoch} iteration #{itr} loss: {loss_value}")

        itr += 1
    
    end = time.time()
    return loss_hist, end, start

In [ ]:
def validate(dataloader, model, device, iou_thresholds):
    valid_image_precision = []
    model.eval()
    with torch.no_grad():  # Gradient calculation will turn off in prediction phase
        for images, targets in dataloader:

            images = list(image.to(device, dtype=torch.float) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            outputs = model(images)
            
    for i, image in enumerate(images):
        boxes = outputs[i]['boxes'].data.cpu().numpy()
        scores = outputs[i]['scores'].data.cpu().numpy()
        gt_boxes = targets[i]['boxes'].cpu().numpy()
        preds_sorted_idx = np.argsort(scores)[::-1]
        preds_sorted = boxes[preds_sorted_idx]
        image_precision = calculate_image_precision(preds_sorted,
                                                        gt_boxes,
                                                        thresholds=iou_thresholds,
                                                        form='coco')
        
        print("precision  : ",image_precision)
        
        valid_image_precision.append(image_precision)

    valid_prec = np.mean(valid_image_precision)
    return valid_prec

# Load FasterRCNN Model

In [ ]:
#https://pytorch.org/vision/stable/generated/torchvision.models.detection.fasterrcnn_resnet50_fpn.html

#model = torchvision.models.resnet18(pretrained=True)

def get_model():
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, min_size=1024)
    
    # one class is pneumonia, and the other is background
    num_classes = 2
    # get the number of input features for the classifier (transfer features fron RPN to ROI pooling)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
   
    return model

In [ ]:
model = get_model()
print(model)

# Hyperparameters

In [ ]:
# # https://github.com/ajinkya98/TensorBoard_PyTorch/blob/master/model.py
# from itertools import product
# # Hyperparameters
# parameters = dict(
#     lr = [0.0001, 0.00001],
#     batch_size = [256, 512],
#     #shuffle = [True, False]
# )
# param_values = [v for v in parameters.values()]

In [ ]:
### https://pytorch.org/docs/stable/optim.html

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# learning parameters
num_epochs = 25
lr = 0.000885
batch_size = 4

model = get_model().to(device)
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr=lr, momentum=0.1, weight_decay=0.0005)
#optimizer = torch.optim.Adam(params, lr=1e-2, betas=(0.9,0.999), eps=1e-08,  weight_decay=0.0)
#optimizer = torch.optim.RMSprop(params, lr=lr,alpha=0.99, weight_decay=0.9,momentum=0.1,centered=False)
# optimizer = torch.optim.SparseAdam(params, lr=lr, betas=(0.9, 0.999), eps=1e-08)
# optimizer = torch.optim.Adamax(params, lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
# optimizer = torch.optim.AdamW(params, lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False, *, maximize=False)
# optimizer = torch.optim.Adagrad(params, lr=lr, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)
# optimizer = torch.optim.Adadelta(params, lr=lr, rho=0.9, eps=1e-06, weight_decay=0)
#optimizer = torch.optim.ASGD(params, lr=lr, lambd=0.0001, alpha=0.9, t0=1000000.0, weight_decay=0.005)
# optimizer = torch.optim.NAdam(params, lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, momentum_decay=0.004)
# optimizer = torch.optim.RAdam(params, lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0
# optimizer = torch.optim.Rprop(params, lr=lr, etas=(0.5, 1.2), step_sizes=(1e-06, 50))


#lr_scheduler = None
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12, gamma=0.1)
                                               

# initialize the Averager
loss_hist = Averager()
iou_thresholds = [x for x in np.arange(0.5, 0.76, 0.05)]

# Training

In [ ]:
train_loss = []
precision = []
for epoch in range(num_epochs):
    itr = 1
    train_loss_hist, end, start = train(train_data_loader, lr_scheduler,
                                        model, optimizer, device,
                                        epoch, loss_hist, itr)
    valid_prec = validate(valid_data_loader, model, device, iou_thresholds)
    print(f"Took {(end-start)/60:.3f} minutes for epoch# {epoch} to train")
    print(f"Epoch #{epoch} Train loss: {train_loss_hist.value}")  
    print(f"Epoch #{epoch} Validation Precision: {valid_prec}")  
    train_loss.append(train_loss_hist.value)
    precision.append(valid_prec)
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

# Saving the model

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn_pneumonia_detection.pth')

# plotting

In [ ]:
# plot the training loss
plt.figure()
plt.plot(train_loss, label='Training loss')
plt.legend()
plt.show()

# plot the validation precision
plt.figure()
plt.plot(precision, label='Validation precision')
plt.legend()
plt.show()

# Tensoboard

# Test model and make predictions

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

images_test_path = 'D:/Datasets/LDA/RSNA/stage_2_test_images'

test_images = os.listdir(images_test_path)
print(f"Validation instances: {len(test_images)}")

In [ ]:
# load a model; pre-trained on COCO
model = get_model()

# os.makedirs('../validation_predictions', exist_ok=True)
model.load_state_dict(torch.load('./fasterrcnn_resnet50_fpn_pneumonia_detection.pth'))
model.to(device)

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], 
                                                             int(j[1][0]), int(j[1][1]), 
                                                             int(j[1][2]), int(j[1][3])))

    return " ".join(pred_strings)

In [ ]:
detection_threshold = 0.8
img_num = 0
results = []
model.eval()
with torch.no_grad():
    for i, image in tqdm(enumerate(test_images), total=len(test_images)):

        orig_image = cv2.imread(f"{images_test_path}/{test_images[i]}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = np.transpose(image, (2, 0, 1)).astype(np.float)
        image = torch.tensor(image, dtype=torch.float).cuda()
        image = torch.unsqueeze(image, 0)

        model.eval()
        cpu_device = torch.device("cpu")

        outputs = model(image)
        
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        if len(outputs[0]['boxes']) != 0:
            for counter in range(len(outputs[0]['boxes'])):
                boxes = outputs[0]['boxes'].data.cpu().numpy()
                scores = outputs[0]['scores'].data.cpu().numpy()
                boxes = boxes[scores >= detection_threshold].astype(np.int32)
                draw_boxes = boxes.copy()
                boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
                
            for box in draw_boxes:
                cv2.rectangle(orig_image,
                            (int(box[0]), int(box[1])),
                            (int(box[2]), int(box[3])),
                            (0, 0, 255), 3)
        
            plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            plt.savefig(f"{test_images[i]}")
            plt.close()
                
            result = {
                'patientId': test_images[i].split('.')[0],
                'PredictionString': format_prediction_string(boxes, scores)
            }
            results.append(result)
        else:
            result = {
                'patientId': test_images[i].split('.')[0],
                'PredictionString': None
            }
            results.append(result)

sub_df = pd.DataFrame(results, columns=['patientId', 'PredictionString'])
print(sub_df.head())
sub_df.to_csv('submission.csv', index=False)